In [2]:
import os
from collections import defaultdict

from plumbum import local
from mgz import header, fast, enums


In [3]:
filename = "../saves/AgeIIDE_Replay_99469529.aoe2record"
ops = []
with local.path(filename).open("rb") as data:
    eof = os.fstat(data.fileno()).st_size
    print("Reading header")
    stream = header.parse_stream(data)
    meta = fast.meta(data)
    print("Finished reading header")
    while data.tell() < eof:
        op = fast.operation(data)
        ops.append(op)

Reading header
Finished reading header


In [4]:
actions_by_player = defaultdict(list)
for op in ops:
    if op[0] == fast.Operation.ACTION:
        action, data = op[1]
        if action in [
            fast.Action.DE_QUEUE,
            fast.Action.BUILD,
            fast.Action.RESEARCH,
        ]:
            actions_by_player[data["player_id"]].append((action, data))

In [5]:
player_id_to_name = {p.player_number: p.name.value for p in stream.de.players if p.player_number >= 0}

In [6]:
# https://halfon.aoe2.se/
unit_id_to_name = {
    83: "Villager",
    93: "Spearman",
    448: "Scout Cavalry"
}

building_id_to_name = {
    68: "Mill",
    562: "Lumber Camp",
    50: "Farm",
    12: "Barracks",
    584: "Mining Camp",
    103: "Blacksmith",
    101: "Stable",
    70: "House",
    84: "Market",
    804: "Palisade Gate",
    # This isn't a mistake, there's multiple archery ranges with different stats :/
    14: "Archery Range",
    87: "Archery Range",
}

technology_id_to_name = {
    101: "Feudal Age",
    22: "Loom",
    202: "Double Bit Axe",
    213: "Wheelbarrow",
    435: "Bloodlines",
    14: "Horse Collar"
}

def get_consecutive_count(values):
    ret = []
    current = None
    count = 0
    for v in values:
        if current != v:
            if count > 0:
                ret.append((current, count))
            current = v
            count = 1
        else:
            count += 1
    return ret


def get_build_order(actions):
    bo = []
    for action, data in actions:
        if action == fast.Action.DE_QUEUE:
            unit_id = data['unit_id']
            bo.append(unit_id_to_name.get(unit_id, unit_id))
        elif action == fast.Action.BUILD:
            building_id = data['building_id']
            bo.append(building_id_to_name.get(building_id, building_id))
        elif action == fast.Action.RESEARCH:
            technology_id = data['technology_id']
            bo.append(technology_id_to_name.get(technology_id, technology_id))
    return get_consecutive_count(bo)


In [7]:
build_orders_by_player = {player_id: get_build_order(action) for player_id, action in actions_by_player.items()}

In [8]:
from rich.console import Console
from rich.table import Table

console = Console(width=200)

table = Table(show_header=True, header_style="bold")

for id, player in player_id_to_name.items():
    table.add_column(player.decode(), no_wrap=True, min_width=15)

def format_bo_tuple(name, count):
    if count == 1:
        return f"{name}"
    return f"{name} x{count}"

for i in range(20):
    table.add_row(
        *[format_bo_tuple(*build_orders_by_player[p][i]) for p in player_id_to_name.keys()]
    )

console.print(table)

┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ TZWC-Jonathan.Franco ┃ L DUBBA E       ┃ TZWC-Guilobo13  ┃ kymcdonald      ┃ TZWC-Expresso48 ┃ DanTheMan       ┃ TZWC-dowglasfranco ┃ MONEY SHOTT     ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ House x2             │ House x2        │ House x2        │ House x2        │ Villager x2     │ Villager x5     │ House x2           │ Loom            │
│ Villager x9          │ Villager x6     │ Loom            │ Villager x5     │ House x2        │ Lumber Camp     │ Villager x11       │ Villager x2     │
│ Lumber Camp          │ Lumber Camp     │ Villager x13    │ Lumber Camp     │ Villager x11    │ Villager x9     │ Lumber Camp        │ House           │
│ Villager x2          │ Villager x3     │ Lumber Camp     │ Villager x4     │ Mill            │ Mill            │ Villager           │ Villager        │
│ House                │ Double Bit Axe  │ House x3        │ House           │ Villager        │ Villager x5     │ Mill               │ House x2        │
│ Mill                 │ Villager x2     │ Villager x2     │ Villager        │ Lumber Camp     │ Barracks        │ Villager           │ Mill            │
│ Villager x2          │ House           │ Mill            │ Mill            │ Villager x8     │ Feudal Age      │ Loom               │ Villager x8     │
│ House x5             │ Loom            │ Villager x3     │ House           │ House x3        │ Farm            │ Villager x2        │ Lumber Camp     │
│ Villager x5          │ Villager x2     │ House           │ Loom            │ Feudal Age      │ 792             │ House x2           │ Villager x6     │
│ Barracks             │ Mill            │ Villager x3     │ Villager x7     │ Villager x2     │ Stable          │ Villager x2        │ House           │
│ House x3             │ Feudal Age      │ Barracks        │ Farm            │ House x5        │ Spearman x2     │ Barracks           │ Villager x6     │
│ Villager x3          │ Lumber Camp     │ Villager x6     │ Villager        │ Villager x4     │ Double Bit Axe  │ Villager x5        │ Mining Camp     │
│ Mining Camp          │ Villager        │ Mining Camp     │ Lumber Camp     │ Market          │ Villager x3     │ Feudal Age         │ Villager        │
│ Villager             │ Barracks        │ Feudal Age      │ Farm            │ Villager x3     │ Bloodlines      │ Mining Camp        │ Mining Camp     │
│ Feudal Age           │ Villager x2     │ 792             │ Barracks        │ Blacksmith      │ Scout Cavalry   │ Villager x2        │ House x8        │
│ Villager x2          │ Stable          │ Villager x4     │ Feudal Age      │ Mining Camp     │ Blacksmith      │ House x3           │ Feudal Age      │
│ Farm x9              │ Villager        │ Lumber Camp     │ Villager        │ Villager x7     │ Scout Cavalry   │ Palisade Gate      │ Barracks x2     │
│ Market               │ House           │ Villager        │ House x2        │ Horse Collar    │ Farm            │ Villager           │ Farm x4         │
│ Blacksmith           │ Blacksmith      │ Farm x3         │ Villager x3     │ Mining Camp     │ Scout Cavalry   │ Double Bit Axe     │ Wheelbarrow     │
│ Villager x5          │ Scout Cavalry   │ Market          │ Mining Camp     │ House x3        │ Stable          │ Archery Range      │ Villager x2     │
└──────────────────────┴─────────────────┴─────────────────┴─────────────────┴─────────────────┴─────────────────┴────────────────────┴─────────────────┘

In [154]:
a = [o for o in ops if o[1][0] == fast.Action.RESEARCH]